In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import os

# For clean plots in Jupyter
%matplotlib inline

# Set random seed
np.random.seed(42)


In [33]:
retail_data = pd.read_excel(r'C:\Users\reddy\Downloads\customer_segmentation_analysis\data\raw\Online_Retail.xlsx')
# Basic check
print(f"Dataset shape: {retail_data.shape}")
retail_data.head()


Dataset shape: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [34]:
# Create directories if they don't exist
os.makedirs(r"C:\Users\reddy\Downloads\customer_segmentation_analysis\process_documentation\screenshots", exist_ok=True)

plt.figure(figsize=(12, 6))
plt.table(cellText=retail_data.head().values, 
          colLabels=retail_data.columns, 
          cellLoc='center', 
          loc='center')
plt.axis('off')

plt.savefig(r"C:\Users\reddy\Downloads\customer_segmentation_analysis\process_documentation\screenshots\data_head.png")
plt.close()


In [35]:
print("Missing values in each column:")
missing_values = retail_data.isnull().sum()
print(missing_values)

plt.figure(figsize=(10, 6))
sns.heatmap(retail_data.isnull(), yticklabels=False, cbar=False, cmap='viridis')
plt.title('Missing Value Heatmap')
plt.tight_layout()
plt.savefig(r'C:\Users\reddy\Downloads\customer_segmentation_analysis\process_documentation\screenshots\missing_values.png')
plt.close()


Missing values in each column:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [13]:
os.makedirs('C:/Users/reddy/Downloads/customer_segmentation_analysis/process_documentation/code_snippets', exist_ok=True)

with open('C:/Users/reddy/Downloads/customer_segmentation_analysis/process_documentation/code_snippets/initial_data_stats.md', 'w') as f:
    f.write(f"# Initial Data Statistics\n\n")
    f.write(f"- Total rows: {retail_data.shape[0]}\n")
    f.write(f"- Total columns: {retail_data.shape[1]}\n")
    f.write(f"- Date range: {retail_data['InvoiceDate'].min()} to {retail_data['InvoiceDate'].max()}\n")
    f.write(f"- Number of unique customers: {retail_data['CustomerID'].nunique()}\n")
    f.write(f"- Number of unique products: {retail_data['StockCode'].nunique()}\n")


In [39]:
count_before = len(retail_data)

# Remove rows with missing CustomerID and negative Quantity
retail_data = retail_data.dropna(subset=['CustomerID'])
retail_data = retail_data[retail_data['Quantity'] > 0]

count_after = len(retail_data)

with open('C:/Users/reddy/Downloads/customer_segmentation_analysis/process_documentation/code_snippets/data_cleaning.md', 'w') as f:
    f.write(f"# Data Cleaning Steps\n\n")
    f.write(f"1. Removed rows with missing CustomerID\n")
    f.write(f"2. Removed rows with negative Quantity (cancelled orders)\n\n")
    f.write(f"- Records before cleaning: {count_before}\n")
    f.write(f"- Records after cleaning: {count_after}\n")
    f.write(f"- Records removed: {count_before - count_after} ({(count_before - count_after)/count_before*100:.2f}%)\n")


In [41]:
retail_data['InvoiceDate'] = pd.to_datetime(retail_data['InvoiceDate'])
retail_data['TotalValue'] = retail_data['Quantity'] * retail_data['UnitPrice']
max_date = retail_data['InvoiceDate'].max()
print(f"Latest purchase date: {max_date}")


Latest purchase date: 2011-12-09 12:50:00


In [19]:
print(retail_data.columns.tolist())


['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


In [43]:
retail_data.columns = retail_data.columns.str.strip()

In [45]:
rfm = retail_data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (max_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalValue': 'sum'
})
rfm.columns = ['Recency', 'Frequency', 'Monetary']
print("\nRFM Metrics - Top 5 customers:")
rfm.head()


RFM Metrics - Top 5 customers:


,Recency,Frequency,Monetary
CustomerID,,,
12346.0,325,1,77183.60
12347.0,1,7,4310.00
12348.0,74,4,1797.24
12349.0,18,1,1757.55
12350.0,309,1,334.40


In [47]:
with open('C:/Users/reddy/Downloads/customer_segmentation_analysis/process_documentation/code_snippets/rfm_calculation.md', 'w') as f:
    f.write(f"# RFM Metrics Calculation\n\n")
    f.write(f"## Definition:\n")
    f.write(f"- **Recency**: Days since customer's last purchase (from {max_date})\n")
    f.write(f"- **Frequency**: Number of unique invoices\n")
    f.write(f"- **Monetary**: Total money spent\n\n")
    f.write(f"## Sample:\n```\n{rfm.head().to_string()}\n```\n")


In [49]:
rfm['AvgPurchaseValue'] = rfm['Monetary'] / rfm['Frequency']

order_values = retail_data.groupby(['CustomerID', 'InvoiceNo'])['TotalValue'].sum().reset_index()
customer_std = order_values.groupby('CustomerID')['TotalValue'].std().fillna(0)
rfm['PurchaseVariability'] = customer_std


In [51]:
with open('C:/Users/reddy/Downloads/customer_segmentation_analysis/process_documentation/code_snippets/feature_engineering.md', 'w') as f:
    f.write(f"# Additional Feature Engineering\n\n")
    f.write(f"1. **Average Purchase Value**: Monetary / Frequency\n")
    f.write(f"2. **Purchase Variability**: Std dev of transaction values\n\n")
    f.write(f"## Sample:\n```\n{rfm[['AvgPurchaseValue', 'PurchaseVariability']].head().to_string()}\n```\n")


In [53]:
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
sns.histplot(rfm['Recency'], kde=True)
plt.title('Distribution of Recency')

plt.subplot(1, 3, 2)
sns.histplot(rfm['Frequency'], kde=True)
plt.title('Distribution of Frequency')

plt.subplot(1, 3, 3)
sns.histplot(rfm['Monetary'], kde=True)
plt.title('Distribution of Monetary')

plt.tight_layout()
plt.savefig('C:/Users/reddy/Downloads/customer_segmentation_analysis/process_documentation/analysis_outputs/rfm_distributions.png')
plt.savefig('C:/Users/reddy/Downloads/customer_segmentation_analysis/visualizations/rfm_distributions.png')
plt.close()


In [55]:
plt.figure(figsize=(10, 8))
sns.heatmap(rfm.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Between RFM Features')
plt.savefig('C:/Users/reddy/Downloads/customer_segmentation_analysis/process_documentation/analysis_outputs/feature_correlations.png')
plt.savefig('C:/Users/reddy/Downloads/customer_segmentation_analysis/visualizations/rfm_correlation.png')
plt.close()


In [59]:
os.makedirs('C:/Users/reddy/Downloads/customer_segmentation_analysis/data/processed', exist_ok=True)
rfm.reset_index().to_csv('C:/Users/reddy/Downloads/customer_segmentation_analysis/data/processed/rfm_data_for_tableau.csv', index=False)


In [61]:
rfm_scaled = rfm.copy()
scaler = StandardScaler()
rfm_scaled[rfm.columns] = scaler.fit_transform(rfm[rfm.columns])

rfm_scaled.reset_index().to_csv('C:/Users/reddy/Downloads/customer_segmentation_analysis/data/processed/rfm_scaled_data.csv', index=False)
